In [ ]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from keras import __version__
import random

from google.colab import drive
drive.mount('/content/drive')
print(__version__)

Mounted at /content/drive
2.4.3


In [ ]:
print(tf.test.gpu_device_name())
def config_GPU():
 !ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
 !pip install gputil
 !pip install psutil
 !pip install humanize
 import psutil
 import humanize
 import os
 import GPUtil as GPU
 GPUs = GPU.getGPUs()
 # XXX: only one GPU on Colab and isn’t guaranteed
 gpu = GPUs[0]
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
config_GPU()
!unzip "/content/drive/My Drive/CN_TP1/dataset.zip" 
!ls

Streaming output truncated to the last 5000 lines.
  inflating: train/TOUCHAN/081.jpg   
  inflating: train/TOUCHAN/082.jpg   
  inflating: train/TOUCHAN/083.jpg   
  inflating: train/TOUCHAN/084.jpg   
  inflating: train/TOUCHAN/085.jpg   
  inflating: train/TOUCHAN/086.jpg   
  inflating: train/TOUCHAN/087.jpg   
  inflating: train/TOUCHAN/088.jpg   
  inflating: train/TOUCHAN/089.jpg   
  inflating: train/TOUCHAN/090.jpg   
  inflating: train/TOUCHAN/091.jpg   
  inflating: train/TOUCHAN/092.jpg   
  inflating: train/TOUCHAN/093.jpg   
  inflating: train/TOUCHAN/094.jpg   
  inflating: train/TOUCHAN/095.jpg   
  inflating: train/TOUCHAN/096.jpg   
  inflating: train/TOUCHAN/097.jpg   
  inflating: train/TOUCHAN/098.jpg   
  inflating: train/TOUCHAN/099.jpg   
  inflating: train/TOUCHAN/100.jpg   
  inflating: train/TOUCHAN/101.jpg   
  inflating: train/TOUCHAN/102.jpg   
  inflating: train/TOUCHAN/103.jpg   
  inflating: train/TOUCHAN/104.jpg   
  inflating: train/TOUCHAN/105.jpg   

In [ ]:
import pathlib
data_dir_train = pathlib.Path('/content/train')
data_dir_test = pathlib.Path('/content/test')
data_dir_valid = pathlib.Path('/content/valid')

In [ ]:
class Settings:
  
  def __init__(self,type,dense_num,dense_val,batch_size,learn_rate,learn_rate_reduce):
    self.type = type
    self.dense_num = dense_num
    self.dense_val = dense_val
    self.batch_size = batch_size
    self.learn_rate = learn_rate
    self.learn_rate_reduce = learn_rate_reduce
  
  def getType(self):
    return self.type
  
  def setType(self,type):
    self.type = type
  def setDense_num(self,dense_num):
    self.dense_num = dense_num
  def setDense_val(self,dense_val):
    self.dense_val = dense_val
  def setBatch_size(self,batch_size):
    self.batch_size = batch_size
  def setLearn_rate(self,learn_rate):
    self.learn_rate = learn_rate
  def setLearn_rate_reduce(self,learn_rate_reduce):
    self.learn_rate_reduce = learn_rate_reduce

  def print(self):
    print("Type: ",self.type,",Dense_Num: ", self.dense_num,",Dense_Val: ", self.dense_val,",Batch_Size: ", self.batch_size,",Learn_rate: ", self.learn_rate,",Learn_rate_reduce: ", self.learn_rate_reduce)

In [ ]:
class CNN:
  def __init__(self,Settings,fitness):
    self.Settings = Settings
    self.fitness = fitness
  def getSettings(self):
    return self.Settings
  def getFitness(self):
    return self.fitness
  def print(self):
    print("Type: ",self.Settings.type,",Dense_Num: ", self.Settings.dense_num,",Dense_Val: ", self.Settings.dense_val,",Batch_Size: ", self.Settings.batch_size,",Learn_rate: ", self.Settings.learn_rate,", Learn_rate_reduce: ", self.Settings.learn_rate_reduce,", Fitness: ",self.fitness)
  

In [ ]:
def startVGG16(epochs,Settings):
  from keras.preprocessing.image import ImageDataGenerator
  vgg16 = keras.applications.vgg16

  datagen = ImageDataGenerator(preprocessing_function=vgg16.preprocess_input,
                               horizontal_flip=True,
                               rescale=1./255,
                               )
  
  train_dataset = datagen.flow_from_directory(data_dir_train,
                                            target_size=(224,224),
                                            batch_size=Settings.batch_size,
                                            shuffle=True,
                                            class_mode='categorical')

  valid_dataset = datagen.flow_from_directory(data_dir_valid,
                                              target_size=(224,224),
                                              batch_size=Settings.batch_size,
                                              shuffle=True,
                                              class_mode='categorical')
  
  test_dataset = datagen.flow_from_directory(data_dir_test,
                                             target_size=(224,224),
                                             batch_size=Settings.batch_size,
                                             shuffle=True,
                                             class_mode='categorical')
  conv_model = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(224,224,3))

  #es = EarlyStopping(monitor='val_loss',patience=5)
  # flatten the output of the convolutional part: 
  x = keras.layers.Flatten()(conv_model.output)
  # three hidden layers
  for n in range(0,Settings.dense_num):
    x = keras.layers.Dense(Settings.dense_val, activation='relu')(x)
  # final softmax layer with two categories (dog and cat)
  predictions = keras.layers.Dense(250, activation='softmax')(x)
  #x = tf.keras.layers.Dropout(0.1)(x)
  #x = data_augmentation(conv_model.input)

  # creating the full model:
  full_model = keras.models.Model(inputs=conv_model.input, outputs=predictions)
   
  for layer in conv_model.layers:
    layer.trainable = False

  reduceLR = tf.keras.callbacks.ReduceLROnPlateau(factor = 0.1, epochs = 4, monitor = 'val_loss')

  full_model.compile(loss='categorical_crossentropy',
                    optimizer=keras.optimizers.Adamax(lr=Settings.learn_rate),
                    metrics=['acc'])
  if Settings.learn_rate_reduce == 'yes':
   history = full_model.fit(
      train_dataset, 
      validation_data = valid_dataset,
      epochs=epochs,
      callbacks=[reduceLR],
   )
  else :
   history = full_model.fit(
      train_dataset, 
      validation_data = valid_dataset,
      epochs=epochs,
   ) 

  Score = full_model.evaluate(test_dataset,verbose=0)
  modelo = CNN(Settings,Score[1]) 
  modelo.print()
  return modelo

In [ ]:
def startVGG19(epochs,Settings):
  from keras.preprocessing.image import ImageDataGenerator
  vgg19 = keras.applications.vgg19

  datagen = ImageDataGenerator(preprocessing_function=vgg19.preprocess_input,
                               horizontal_flip=True,
                               validation_split=0.1,
                               rescale=1./255,
                               )
  
  train_dataset = datagen.flow_from_directory(data_dir_train,
                                            target_size=(224,224),
                                            batch_size=Settings.batch_size,
                                            shuffle=True,
                                            class_mode='categorical')

  valid_dataset = datagen.flow_from_directory(data_dir_valid,
                                              target_size=(224,224),
                                              batch_size=Settings.batch_size,
                                              shuffle=True,
                                              class_mode='categorical')
  
  test_dataset = datagen.flow_from_directory(data_dir_test,
                                             target_size=(224,224),
                                             batch_size=Settings.batch_size,
                                             shuffle=True,
                                             class_mode='categorical')
  
  conv_model = vgg19.VGG19(weights='imagenet', include_top=False, input_shape=(224,224,3))

  #es = EarlyStopping(monitor='val_loss',patience=5)
  # flatten the output of the convolutional part: 
  x = keras.layers.Flatten()(conv_model.output)
  # three hidden layers
  for n in range(0,Settings.dense_num):
    x = keras.layers.Dense(Settings.dense_val, activation='relu')(x)
  # final softmax layer with two categories (dog and cat)
  predictions = keras.layers.Dense(250, activation='softmax')(x)
  #x = tf.keras.layers.Dropout(0.1)(x)
  #x = data_augmentation(conv_model.input)

  # creating the full model:
  full_model = keras.models.Model(inputs=conv_model.input, outputs=predictions)
   
  for layer in conv_model.layers:
    layer.trainable = False

  reduceLR = tf.keras.callbacks.ReduceLROnPlateau(factor = 0.1, epochs = 4, monitor = 'val_loss')

  full_model.compile(loss='categorical_crossentropy',
                    optimizer=keras.optimizers.Adamax(lr=Settings.learn_rate),
                    metrics=['acc'])
  if Settings.learn_rate_reduce == 'yes':
   history = full_model.fit(
      train_dataset, 
      validation_data = valid_dataset,
      epochs=epochs,
      callbacks=[reduceLR],
   )
  else :
   history = full_model.fit(
      train_dataset, 
      validation_data = valid_dataset,
      epochs=epochs,
   ) 

  Score = full_model.evaluate(test_dataset,verbose=0)
  modelo = CNN(Settings,Score[1]) 
  modelo.print()
  return modelo

In [ ]:
def startResnet(epochs,Settings):
  from keras.preprocessing.image import ImageDataGenerator
  resnet50 = keras.applications.resnet50

  datagen = ImageDataGenerator(preprocessing_function=resnet50.preprocess_input,
                               horizontal_flip=True,
                               validation_split=0.1,
                               rescale=1./255,
                               )
  
  train_dataset = datagen.flow_from_directory(data_dir_train,
                                            target_size=(224,224),
                                            batch_size=Settings.batch_size,
                                            shuffle=True,
                                            class_mode='categorical')

  valid_dataset = datagen.flow_from_directory(data_dir_valid,
                                              target_size=(224,224),
                                              batch_size=Settings.batch_size,
                                              shuffle=True,
                                              class_mode='categorical')
  test_dataset = datagen.flow_from_directory(data_dir_test,
                                             target_size=(224,224),
                                             batch_size=Settings.batch_size,
                                             shuffle=True,
                                             class_mode='categorical')
  conv_model = resnet50.ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3))

  #es = EarlyStopping(monitor='val_loss',patience=5)
  # flatten the output of the convolutional part: 
  x = keras.layers.Flatten()(conv_model.output)
  # three hidden layers
  for n in range(0,Settings.dense_num):
    x = keras.layers.Dense(Settings.dense_val, activation='relu')(x)
  # final softmax layer with two categories (dog and cat)
  predictions = keras.layers.Dense(250, activation='softmax')(x)
  #x = tf.keras.layers.Dropout(0.1)(x)
  #x = data_augmentation(conv_model.input)

  # creating the full model:
  full_model = keras.models.Model(inputs=conv_model.input, outputs=predictions)
   
  for layer in conv_model.layers:
    layer.trainable = False

  reduceLR = tf.keras.callbacks.ReduceLROnPlateau(factor = 0.1, epochs = 4, monitor = 'val_loss')

  full_model.compile(loss='categorical_crossentropy',
                    optimizer=keras.optimizers.Adamax(lr=Settings.learn_rate),
                    metrics=['acc'])
  
  if Settings.learn_rate_reduce == 'yes':
   history = full_model.fit(
      train_dataset, 
      validation_data = valid_dataset,
      epochs=epochs,
      callbacks=[reduceLR],
   )
  else :
   history = full_model.fit(
      train_dataset, 
      validation_data = valid_dataset,
      epochs=epochs,
   ) 
  
  Score = full_model.evaluate(test_dataset,verbose=0)
  modelo = CNN(Settings,Score[1]) 
  modelo.print()
  return modelo

In [ ]:
#Função que seleciona os melhores pais
def mating(list):
  global best_settings
  best_id = 0
  second_best_id = 0
  for i in range(0,elements):
    if list[i].getFitness() > list[second_best_id].getFitness():
     if list[i].getFitness() > list[best_id].getFitness():
       best_id = i 
     else:
       second_best_id = i
  parents = [list[best_id].getSettings(),list[second_best_id].getSettings()]
  #Guardar o melhor pai para na ultima geração se poder utilizar para fine tuning
  best_settings = list[best_id].getSettings()
  best_settings.print()
  return parents

In [ ]:
#Função que gera as possiveis settings dos filhos
def crossover(cnnList):
  settings_list = []
  settings_list.append(Settings(cnnList[0].type,cnnList[1].dense_num,cnnList[0].dense_val,cnnList[1].batch_size,cnnList[0].learn_rate,cnnList[1].learn_rate_reduce))
  settings_list.append(Settings(cnnList[1].type,cnnList[0].dense_num,cnnList[1].dense_val,cnnList[0].batch_size,cnnList[1].learn_rate,cnnList[0].learn_rate_reduce))
  settings_list.append(Settings(cnnList[0].type,cnnList[0].dense_num,cnnList[0].dense_val,cnnList[1].batch_size,cnnList[1].learn_rate,cnnList[1].learn_rate_reduce))
  settings_list.append(Settings(cnnList[1].type,cnnList[1].dense_num,cnnList[1].dense_val,cnnList[0].batch_size,cnnList[0].learn_rate,cnnList[0].learn_rate_reduce))
  return settings_list


In [ ]:
#Função de mutação
def mutation(settings_list):
  for i in range(0,4):
    probability = random.uniform(0.0,1.0)
    if probability < mutation_rate:
     rand_setting = random.randint(0,5)
     if rand_setting == 0:
       settings_list[i].setType(random.choice(['VGG16','VGG19','Resnet']))
     if rand_setting == 1:
       settings_list[i].setDense_num(random.randint(1,4))
     if rand_setting == 2:
       settings_list[i].setDense_val(random.choice([250,512]))
     if rand_setting == 3:
       settings_list[i].setBatch_size(random.choice([32,64]))
     if rand_setting == 4:
       settings_list[i].setLearn_rate(random.choice([0.1,0.01,0.001,0.0001,0.00001]))
     if rand_setting == 5:
       settings_list[i].setLearn_rate_reduce(random.choice(['yes','no']))  
  return settings_list

In [ ]:
import pickle

def init(elements,generations):
  models = []
  mating_cnns = []
  crossover_settings = []
  g = 3
  for g in range(g,generations):
   print("Começou a geração: ",g+1)
   epochs = 5
   if g != 0 : 
    mating_cnns = mating(models)
    crossover_settings = crossover(mating_cnns)
    crossover_settings_mutated = mutation(crossover_settings)
   for n in range(0,elements):
     print("CNN número: ",n+1)
     if g == 0:
      type = random.choice(['VGG16','VGG19','Resnet'])
      dense_num = random.randint(1,4)
      dense_val=random.choice([250,512])
      batch_size=random.choice([32,64])
      learn_rate=random.choice([0.1,0.01,0.001,0.0001,0.00001]) 
      learn_rate_reduce=random.choice(['yes','no'])
      settings = Settings(type,dense_num,dense_val,batch_size,learn_rate,learn_rate_reduce)
     else:
      rand_setting = random.randint(0,3)
      settings = crossover_settings_mutated[rand_setting]
     if settings.getType() == 'VGG16':
       print("Starting VGG16")
       models.append(startVGG16(epochs,settings))
     if settings.getType() == 'VGG19':
       print("Starting VGG19")
       models.append(startVGG19(epochs,settings))
     if settings.getType() == 'Resnet':
       print("Starting Resnet")
       models.append(startResnet(epochs,settings))
     #with open('/content/drive/My Drive/objs.pkl','wb') as f:
      #pickle.dump(models,f) 
     print("-----------------------------")
  print("---------------------------------------------------------")

In [ ]:
#Uma vez que o algoritmo genético apenas devolve os cromossomas ideais, temos de voltar a treinar um modelo

best_settings.print()
if(best_settings.getType() == 'VGG16'):
  from keras.preprocessing.image import ImageDataGenerator
  vgg16 = keras.applications.vgg16

  datagen = ImageDataGenerator(preprocessing_function=vgg16.preprocess_input,
                               horizontal_flip=True,
                               rescale=1./255,
                               )
  
  train_dataset = datagen.flow_from_directory(data_dir_train,
                                            target_size=(224,224),
                                            batch_size=best_settings.batch_size,
                                            shuffle=True,
                                            class_mode='categorical')

  valid_dataset = datagen.flow_from_directory(data_dir_valid,
                                              target_size=(224,224),
                                              batch_size=best_settings.batch_size,
                                              shuffle=True,
                                              class_mode='categorical')
  
  test_dataset = datagen.flow_from_directory(data_dir_test,
                                             target_size=(224,224),
                                             batch_size=best_settings.batch_size,
                                             shuffle=True,
                                             class_mode='categorical')
  conv_model = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(224,224,3))

  #es = EarlyStopping(monitor='val_loss',patience=5)
  # flatten the output of the convolutional part: 
  x = keras.layers.Flatten()(conv_model.output)
  # three hidden layers
  for n in range(0,best_settings.dense_num):
    x = keras.layers.Dense(best_settings.dense_val, activation='relu')(x)
  # final softmax layer with two categories (dog and cat)
  predictions = keras.layers.Dense(250, activation='softmax')(x)
  #x = tf.keras.layers.Dropout(0.1)(x)
  #x = data_augmentation(conv_model.input)

  # creating the full model:
  full_model = keras.models.Model(inputs=conv_model.input, outputs=predictions)
   
  for layer in conv_model.layers:
    layer.trainable = False

  reduceLR = tf.keras.callbacks.ReduceLROnPlateau(factor = 0.1, epochs = 4, monitor = 'val_loss')

  full_model.compile(loss='categorical_crossentropy',
                    optimizer=keras.optimizers.Adamax(lr=best_settings.learn_rate),
                    metrics=['acc'])
  if best_settings.learn_rate_reduce == 'yes':
   history = full_model.fit(
      train_dataset, 
      validation_data = valid_dataset,
      epochs=10,
      callbacks=[reduceLR],
   )
  else :
   history = full_model.fit(
      train_dataset, 
      validation_data = valid_dataset,
      epochs=epochs,
   ) 


Type:  VGG16 ,Dense_Num:  4 ,Dense_Val:  512 ,Batch_Size:  64 ,Learn_rate:  0.001 ,Learn_rate_reduce:  yes
Found 35215 images belonging to 250 classes.
Found 1250 images belonging to 250 classes.
Found 1250 images belonging to 250 classes.
Epoch 1/10
551/551 [==============================] - 105s 189ms/step - loss: 4.7628 - acc: 0.0871 - val_loss: 1.8401 - val_acc: 0.5272
Epoch 2/10
551/551 [==============================] - 103s 187ms/step - loss: 1.8281 - acc: 0.5355 - val_loss: 1.0038 - val_acc: 0.7320
Epoch 3/10
551/551 [==============================] - 103s 186ms/step - loss: 1.1086 - acc: 0.7054 - val_loss: 0.7719 - val_acc: 0.7872
Epoch 4/10
551/551 [==============================] - 103s 187ms/step - loss: 0.8174 - acc: 0.7731 - val_loss: 0.6785 - val_acc: 0.8064
Epoch 5/10
551/551 [==============================] - 103s 187ms/step - loss: 0.6439 - acc: 0.8222 - val_loss: 0.6526 - val_acc: 0.8224
Epoch 6/10
551/551 [==============================] - 103s 187ms/step - loss: 0.

NameError: ignored

In [ ]:
if(best_settings.getType() == 'VGG19'):
  from keras.preprocessing.image import ImageDataGenerator
  vgg19 = keras.applications.vgg19

  datagen = ImageDataGenerator(preprocessing_function=vgg19.preprocess_input,
                               horizontal_flip=True,
                               rescale=1./255,
                               )
  
  train_dataset = datagen.flow_from_directory(data_dir_train,
                                            target_size=(224,224),
                                            batch_size=best_settings.batch_size,
                                            shuffle=True,
                                            class_mode='categorical')

  valid_dataset = datagen.flow_from_directory(data_dir_valid,
                                              target_size=(224,224),
                                              batch_size=best_settings.batch_size,
                                              shuffle=True,
                                              class_mode='categorical')
  
  test_dataset = datagen.flow_from_directory(data_dir_test,
                                             target_size=(224,224),
                                             batch_size=best_settings.batch_size,
                                             shuffle=True,
                                             class_mode='categorical')
  conv_model = vgg19.VGG19(weights='imagenet', include_top=False, input_shape=(224,224,3))

  #es = EarlyStopping(monitor='val_loss',patience=5)
  # flatten the output of the convolutional part: 
  x = keras.layers.Flatten()(conv_model.output)
  # three hidden layers
  for n in range(0,best_settings.dense_num):
    x = keras.layers.Dense(best_settings.dense_val, activation='relu')(x)
  # final softmax layer with two categories (dog and cat)
  predictions = keras.layers.Dense(250, activation='softmax')(x)
  #x = tf.keras.layers.Dropout(0.1)(x)
  #x = data_augmentation(conv_model.input)

  # creating the full model:
  full_model = keras.models.Model(inputs=conv_model.input, outputs=predictions)
   
  for layer in conv_model.layers:
    layer.trainable = False

  reduceLR = tf.keras.callbacks.ReduceLROnPlateau(factor = 0.1, epochs = 4, monitor = 'val_loss')

  full_model.compile(loss='categorical_crossentropy',
                    optimizer=keras.optimizers.Adamax(lr=best_settings.learn_rate),
                    metrics=['acc'])
  if best_settings.learn_rate_reduce == 'yes':
   history = full_model.fit(
      train_dataset, 
      validation_data = valid_dataset,
      epochs=10,
      callbacks=[reduceLR],
   )
  else :
   history = full_model.fit(
      train_dataset, 
      validation_data = valid_dataset,
      epochs=epochs,
   ) 


In [ ]:
if(best_settings.getType() == 'Resnet'):
  from keras.preprocessing.image import ImageDataGenerator
  resnet50 = keras.applications.resnet50

  datagen = ImageDataGenerator(preprocessing_function=vgg16.preprocess_input,
                               horizontal_flip=True,
                               rescale=1./255,
                               )
  
  train_dataset = datagen.flow_from_directory(data_dir_train,
                                            target_size=(224,224),
                                            batch_size=best_settings.batch_size,
                                            shuffle=True,
                                            class_mode='categorical')

  valid_dataset = datagen.flow_from_directory(data_dir_valid,
                                              target_size=(224,224),
                                              batch_size=best_settings.batch_size,
                                              shuffle=True,
                                              class_mode='categorical')
  
  test_dataset = datagen.flow_from_directory(data_dir_test,
                                             target_size=(224,224),
                                             batch_size=best_settings.batch_size,
                                             shuffle=True,
                                             class_mode='categorical')
  conv_model = resnet50.ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3))

  #es = EarlyStopping(monitor='val_loss',patience=5)
  # flatten the output of the convolutional part: 
  x = keras.layers.Flatten()(conv_model.output)
  # three hidden layers
  for n in range(0,best_settings.dense_num):
    x = keras.layers.Dense(best_settings.dense_val, activation='relu')(x)
  # final softmax layer with two categories (dog and cat)
  predictions = keras.layers.Dense(250, activation='softmax')(x)
  #x = tf.keras.layers.Dropout(0.1)(x)
  #x = data_augmentation(conv_model.input)

  # creating the full model:
  full_model = keras.models.Model(inputs=conv_model.input, outputs=predictions)
   
  for layer in conv_model.layers:
    layer.trainable = False

  reduceLR = tf.keras.callbacks.ReduceLROnPlateau(factor = 0.1, epochs = 4, monitor = 'val_loss')

  full_model.compile(loss='categorical_crossentropy',
                    optimizer=keras.optimizers.Adamax(lr=best_settings.learn_rate),
                    metrics=['acc'])
  if best_settings.learn_rate_reduce == 'yes':
   history = full_model.fit(
      train_dataset, 
      validation_data = valid_dataset,
      epochs=10,
      callbacks=[reduceLR],
   )
  else :
   history = full_model.fit(
      train_dataset, 
      validation_data = valid_dataset,
      epochs=epochs,
   ) 


In [ ]:
full_model.trainable = True

full_model.compile(loss='categorical_crossentropy',
              optimizer = tf.keras.optimizers.Adamax(lr=best_settings.learn_rate/10),
              metrics=['acc'])
  
history_fine = full_model.fit(train_dataset,
                                epochs=5,
                                validation_data=valid_dataset)

Epoch 1/5
551/551 [==============================] - 224s 402ms/step - loss: 0.6274 - acc: 0.8655 - val_loss: 0.3520 - val_acc: 0.9080
Epoch 2/5
551/551 [==============================] - 220s 399ms/step - loss: 0.0940 - acc: 0.9717 - val_loss: 0.3753 - val_acc: 0.9088
Epoch 3/5
551/551 [==============================] - 220s 398ms/step - loss: 0.0756 - acc: 0.9760 - val_loss: 0.3830 - val_acc: 0.8992
Epoch 4/5
551/551 [==============================] - 220s 399ms/step - loss: 0.0688 - acc: 0.9775 - val_loss: 0.4211 - val_acc: 0.8960
Epoch 5/5
551/551 [==============================] - 220s 398ms/step - loss: 0.0600 - acc: 0.9809 - val_loss: 0.3205 - val_acc: 0.9216


In [ ]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(25)

score = full_model.evaluate(test_dataset,verbose=0)
print('Test Loss:', score[0])
print('Test accuracy', score[1])

Test Loss: 0.318600594997406
Test accuracy 0.9287999868392944


In [ ]:
global elements
global generations
global mutation_rate
elements = 5
generations = 3
mutation_rate = 0.1
init(elements,generations)

Type:  VGG16 ,Dense_Num:  2 ,Dense_Val:  250 ,Batch_Size:  64 ,Learn_rate:  0.0001 , Learn_rate_reduce:  yes , Fitness:  0.6583999991416931
Type:  VGG16 ,Dense_Num:  4 ,Dense_Val:  512 ,Batch_Size:  64 ,Learn_rate:  0.001 , Learn_rate_reduce:  yes , Fitness:  0.8312000036239624
Type:  VGG16 ,Dense_Num:  4 ,Dense_Val:  512 ,Batch_Size:  32 ,Learn_rate:  0.01 , Learn_rate_reduce:  yes , Fitness:  0.5680000185966492
Type:  Resnet ,Dense_Num:  2 ,Dense_Val:  512 ,Batch_Size:  32 ,Learn_rate:  0.001 , Learn_rate_reduce:  yes , Fitness:  0.004000000189989805
Type:  VGG19 ,Dense_Num:  1 ,Dense_Val:  512 ,Batch_Size:  64 ,Learn_rate:  0.0001 , Learn_rate_reduce:  no , Fitness:  0.7775999903678894
Type:  VGG16 ,Dense_Num:  4 ,Dense_Val:  512 ,Batch_Size:  64 ,Learn_rate:  0.001 ,Learn_rate_reduce:  yes
---------------------------------------------------------
